In [1]:
# Data preparation for Ph.D thesis
# @author: Andres L. Suarez-Cetrulo
%config IPCompleter.greedy=True

In [2]:
# Imports
import pandas as pd
import numpy as np
import datetime
import json

# Using TALib abstract API to create a dictionary of technical indicators to iterate later.
from talib import abstract

In [3]:
def get_datediff(start, end): 
    """ 
    Obtains amount of days between starting and end date. 
    @param: start: start date
    @param: end: end date
    @return: amount of days between both input dates
    """
    d1 = datetime.datetime.strptime(start.replace('-', ':')+':00:00:00', '%Y:%m:%d:%H:%M:%S')
    d2 = datetime.datetime.strptime(end.replace('-', ':')+':23:59:59', '%Y:%m:%d:%H:%M:%S')
    return round((d2 - d1).total_seconds() / 3600 / 24, 0)

In [4]:
# Declaring date ranges for paper neurocrypto
# ==============================================
# test and train
# start_date='2018-07-01'
# end_date='2018-09-30'
#start_date='2018-07-01'
#end_date='2019-06-30'

# devset
#start_date='2018-05-01'
#end_date='2018-06-30'

# test predictions
# start_date='2019-07-01'
# end_date='2019-07-31'
# ==============================================

# Ranges for PhD

# start_date='2018-07-01'  # for mahalanobis matrices
# end_date='2018-09-30'

# start_date='2018-10-01'  # for devset
# end_date='2018-12-31'

start_date='2019-01-01'  # for trainset
end_date='2020-06-30'

# Crypto currencies to take into account 
CURRENCIES_AND_SYMBOLS = {
    'BITCOIN': 'BTC',
    'DASH': 'DASH',
    'ETHEREUM': 'ETH',
    'LITECOIN': 'LTC',
    'MONERO': 'XMR',
    'RIPPLE': 'XRP'
}
SELECTED = 0 # Selected cryptocurrency, from dictionary above (0-INDEXED)

# Paths
DATA_PATH='C:\\Users\\suare\\data\\'
# SOURCE_PATH=DATA_PATH+'crypto_compare\\'
SOURCE_PATH='C:\\Users\\suare\\MEGA\\PhD\\1 Code and data\\data\\raw\\crypto_compare\\'
RESULT_PATH=DATA_PATH+'analysis\\CRYPTO\\'

# select crypto currency to be processed
CURRENCY=list(CURRENCIES_AND_SYMBOLS.keys())[SELECTED]
SYMBOL=CURRENCIES_AND_SYMBOLS[CURRENCY]
INPUT_FILE_EXTENSION='.csv.gz'
print(SYMBOL)

# List of dates for files to be loaded ( +1 in periods is extra day in next month so the labels are complete)
FILES = pd.date_range(start_date, periods=(get_datediff(start_date, end_date) + 1), freq='D')
RESULT_FILEPATH_PREPROCESSED=RESULT_PATH+CURRENCY+'_['+str(FILES[0])[:10]+'_to_'+str(FILES[len(FILES)-1])[:10]+'].csv'
RESULT_FILEPATH_PROCESSED=RESULT_PATH+CURRENCY+'_('+start_date+'_to_'+end_date+')_indicators.csv'

# Creating a dictionary of technical indicators using TALib abstract API
indicator = {}

# Add as many indicators as necessary: see all indicators in https://mrjbq7.github.io/ta-lib/
indicator['sma']=abstract.Function('sma') # Simple Moving Average
indicator['ema']=abstract.Function('ema') # Exponential Moving Average
# indicator['wma']=abstract.Function('wma') # Weighted Moving Average
indicator['mom']=abstract.Function('mom') # Momentum
indicator['stoch']=abstract.Function('stoch') # Stochastic (returns K and D)
indicator['macd']=abstract.Function('macd') # Moving Average Convergence/Divergence
indicator['rsi']=abstract.Function('rsi') # Relative Strength Index
indicator['willr']=abstract.Function('willr') # Williams' %R
indicator['adosc']=abstract.Function('adosc') # Chaikin A/D Oscillator
indicator['cci']=abstract.Function('cci') # Commodity Channel Index

# other indicators
#indicator['adx']=abstract.Function('adx') # Average Directional Movement Index
#indicator['aroon']=abstract.Function('aroon') # Aroon
#indicator['bbands']=abstract.Function('bbands') # Bollinger Bands
#indicator['obv']=abstract.Function('obv') # On Balance Volume
#indicator['trima']=abstract.Function('trima') # Triangular Moving Average
#indicator['roc']=abstract.Function('roc') # Rate of change : ((price/prevPrice)-1)*100
#indicator['rocr']=abstract.Function('rocr') # Rate of change ratio: (price/prevPrice)
#indicator['stochf']=abstract.Function('stochf') # Stochastic fast (returns K and D)
#indicator['adosc']=abstract.Function('adosc') # Chaikin A/D Oscillator
#indicator['medprice']=abstract.Function('medprice') # Median Price
#indicator['typprice']=abstract.Function('typprice') # Typical Price
#indicator['wclprice']=abstract.Function('wclprice') # Weighted Close Price
#indicator['atr']=abstract.Function('atr') # Average True Range
#indicator['macdfix']=abstract.Function('macdfix') # #Moving Average Convergence/Divergence Fix 12/26
#indicator['mfi']=abstract.Function('mfi') # Money Flow Index


BTC


In [5]:
# In the current datasets:

##############################
#
# “Volume From” and “To” are the volumes of the respective currency pair.
#
# For example, for the BTC-USD pair  
#
# “Volume From” is the number of Bitcoins traded for US dollars 
# “Volume To” is the number of dollars traded (for the period) for Bitcoins.
#
#
# In summary:
# -----------
# "volumeto" means the volume in the currency that is being traded
# "volumefrom" means the volume in the base currency that things are traded into.
#
# As all our cryptocurrencies are Currency/USD, then we use volumeto (volume in dollars) as volume
# This is consistent with the other columns ohcl, that are in dollars also.
#
##############################

# picking volume_to as actual volume
time_column='timestamp'
columns = ['close','high','low','open','time','volumefrom','volume',time_column] 

In [6]:
# Create list of DFs
dataframes=[]
aux_df= pd.DataFrame(columns=columns)

# Iterating through files
for file in FILES:
    print(file)
    file = str(file)[:10] # exclude time (00:00:00)
    file_path = CURRENCY+'/'+SYMBOL+'_'+file+INPUT_FILE_EXTENSION
    csv_path=SOURCE_PATH+file_path
    try:
        new_df=pd.read_csv(csv_path, compression='gzip', sep=';', \
                           parse_dates=True,infer_datetime_format=True, \
                           index_col = 0)
        new_df.timestamp = new_df.timestamp.astype('datetime64[ns]')
    except:
        # If no csv is found, search for a JSON
        file_path = f"{CURRENCY}/{file.replace('-', '')}_{SYMBOL}.json"
        csv_path=SOURCE_PATH+file_path
        print(csv_path)
        data = json.load(open(csv_path))
        new_df = pd.DataFrame(data['Data'])
        new_df['timestamp'] = pd.to_datetime(new_df.time, unit='s')
    new_df.columns = columns
    #dataframes.append(new_df)
    
    # BEGIN-NEW!
    # Removing from overall dataframe all rows captured the next day 
    #  This works as a drop_duplicates, but it will also apply corrections in prices/volumes 
    dups=pd.merge(new_df.reset_index(drop=True),aux_df.reset_index(drop=True), on=[time_column])[time_column]
    if len(dups)>1: aux_df=aux_df[~aux_df.timestamp.isin(dups)]
    new_df.index=new_df[time_column]
    new_df.index=pd.to_datetime(new_df.index)
    aux_df.index=aux_df[time_column]
    aux_df.index=pd.to_datetime(aux_df.index)
    dataframes=[]
    dataframes.append(aux_df) # only add clean DF
    dataframes.append(new_df) # add new data
    aux_df=pd.concat(dataframes) # update dataframe to check for dups
    # END-NEW!

# concat all dataframes in a single one
#df=pd.concat(dataframes) # this shouldn't be used anymore as it would include the dups removed above
#df=df.drop_duplicates(['close','high','low','open','time','volumefrom','volume','timestamp']) # 21087 rows without duplicates
df = aux_df # dropping duplicates and selecting second occurrence of a row if there is a correction (done in the loop above)

# Index dataframe by its actual readable timestamp
df.index=df['timestamp']
df.index=pd.to_datetime(df.index)
df=df.sort_index()

# Save raw concatenated file in analysis
df.to_csv(RESULT_FILEPATH_PREPROCESSED, sep=';', encoding='utf-8', index = True)
print(df.head())

2019-01-01 00:00:00
2019-01-02 00:00:00
2019-01-03 00:00:00
2019-01-04 00:00:00
2019-01-05 00:00:00
2019-01-06 00:00:00
2019-01-07 00:00:00
2019-01-08 00:00:00
2019-01-09 00:00:00
2019-01-10 00:00:00
2019-01-11 00:00:00
2019-01-12 00:00:00
2019-01-13 00:00:00
2019-01-14 00:00:00
2019-01-15 00:00:00
2019-01-16 00:00:00
2019-01-17 00:00:00
2019-01-18 00:00:00
2019-01-19 00:00:00
2019-01-20 00:00:00
2019-01-21 00:00:00
2019-01-22 00:00:00
2019-01-23 00:00:00
2019-01-24 00:00:00
2019-01-25 00:00:00
2019-01-26 00:00:00
2019-01-27 00:00:00
2019-01-28 00:00:00
2019-01-29 00:00:00
2019-01-30 00:00:00
2019-01-31 00:00:00
2019-02-01 00:00:00
2019-02-02 00:00:00
2019-02-03 00:00:00
2019-02-04 00:00:00
2019-02-05 00:00:00
2019-02-06 00:00:00
2019-02-07 00:00:00
2019-02-08 00:00:00
2019-02-09 00:00:00
2019-02-10 00:00:00
2019-02-11 00:00:00
2019-02-12 00:00:00
2019-02-13 00:00:00
2019-02-14 00:00:00
2019-02-15 00:00:00
2019-02-16 00:00:00
2019-02-17 00:00:00
2019-02-18 00:00:00
2019-02-19 00:00:00


2020-01-28 00:00:00
2020-01-29 00:00:00
2020-01-30 00:00:00
2020-01-31 00:00:00
2020-02-01 00:00:00
2020-02-02 00:00:00
2020-02-03 00:00:00
2020-02-04 00:00:00
2020-02-05 00:00:00
2020-02-06 00:00:00
2020-02-07 00:00:00
2020-02-08 00:00:00
2020-02-09 00:00:00
2020-02-10 00:00:00
2020-02-11 00:00:00
2020-02-12 00:00:00
2020-02-13 00:00:00
2020-02-14 00:00:00
2020-02-15 00:00:00
2020-02-16 00:00:00
2020-02-17 00:00:00
2020-02-18 00:00:00
2020-02-19 00:00:00
2020-02-20 00:00:00
2020-02-21 00:00:00
2020-02-22 00:00:00
2020-02-23 00:00:00
2020-02-24 00:00:00
2020-02-25 00:00:00
2020-02-26 00:00:00
2020-02-27 00:00:00
2020-02-28 00:00:00
2020-02-29 00:00:00
2020-03-01 00:00:00
2020-03-02 00:00:00
2020-03-03 00:00:00
2020-03-04 00:00:00
2020-03-05 00:00:00
2020-03-06 00:00:00
2020-03-07 00:00:00
2020-03-08 00:00:00
2020-03-09 00:00:00
2020-03-10 00:00:00
2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-13 00:00:00
2020-03-14 00:00:00
2020-03-15 00:00:00
2020-03-16 00:00:00
2020-03-17 00:00:00


In [7]:
RESULT_FILEPATH_PREPROCESSED

'C:\\Users\\suare\\data\\analysis\\CRYPTO\\BITCOIN_[2019-01-01_to_2020-07-01].csv'

In [153]:
# file_path = f"{CURRENCY}/20191004_{SYMBOL}.json"
# csv_path=SOURCE_PATH+file_path
# print(csv_path)
# data = json.load(open(csv_path))
# df = pd.DataFrame(data['Data'])
# df['timestamp'] = pd.to_datetime(df.time, unit='s')
# df.dtypes

In [154]:
#         new_df=pd.read_csv(csv_path, compression='gzip', sep=';', \
#                            parse_dates=True,infer_datetime_format=True, \
#                            index_col = 0)
#         new_df.timestamp = new_df.timestamp.astype('datetime64[ns]')
#         new_df

In [155]:
# print(start_date)
# print(end_date)
# print(get_datediff(start_date, end_date)*60*24)
# print(start_date+' 00:00:00')
# start_time = datetime.datetime.strptime(start_date+' 00:00:00', '%Y-%m-%d %H:%M:%S')
# end_time = datetime.datetime.strptime(end_date+' 23:59:59', '%Y-%m-%d %H:%M:%S')
# print(len(df.reset_index(drop=True)[(df.reset_index(drop=True)['timestamp'] >= start_date+' 00:00:00') & (df.reset_index(drop=True)['timestamp'] <= end_date+" 23:59:59")]))
# # Testing the dataset (TO-DO: use a proper unit testing library)

In [8]:
#First check to see if there are any gaps or duplicates.
#The number of rows must be diff*60*24 in df[(df['timestamp'] > start_date) & (df['timestamp'] <= end_date+" 23:59:59")
test1=(get_datediff(start_date, end_date)*60*24) == len(df.reset_index(drop=True)[(df.reset_index(drop=True)['timestamp'] >= start_date+' 00:00:00') & (df.reset_index(drop=True)['timestamp'] <= end_date+" 23:59:59")])
print ('TEST 1: '+ str(test1) + ' - The amount of records is correct?')

# Second check to see if there are any gaps
ohlc_dict = {'open':'first','high':'max','low':'min','close':'last','volume':'sum'}
test=df.resample('1min').agg(ohlc_dict) # This will create rows with NaN values if there are any minutes missing in the timerange
test[test['open'].isnull()] # only displays gaps. if empty, happy days!
test2=len(test[test['open'].isnull()]) == 0
print ('TEST 2: '+str(test2)+' - Is the dataset complete? (no gaps)')

test3=len(df)==len(test)
print ('TEST 3: '+str(test3)+' - Are the resampled and the original dataset of the same size? (so # rows is ok)')

test4= df.reset_index(drop=True).groupby(['timestamp']).count()['high']
test4=(len(test4[test4 > 1])==0)
print('TEST 4: '+str(test4)+' - Is the dataset free from any duplicates?')

# If there are any dups (Test 4 == False), check the given row (change timestamp below)
# example=df[(df['timestamp'] == '2018-07-01 20:33:00')]
# example

print('')
print('ONLY CONTINUE IF ALL ARE TRUE!!')
print('--------------------------------')
if not (test1 and test2 and test3 and test4):
    df = [] 
    indicators = []
    print("OOOOOOOPS!! Your next thing won't work! :-)")
    print("Fix this mess first!")
else: print("All true. Please, continue. :-)")

#If there were any gaps (TEST 2 = False), then fill them:
#test['volume'] = test['volume'].fillna(0) 
#test['close'] = test['close'].ffill()     
#test['open'] = test['open'].fillna(test['close']) 
#test['low'] = test['low'].fillna(test['close']) 
#test['high'] = test['high'].fillna(test['close'])
#df = test

TEST 1: True - The amount of records is correct?
TEST 2: True - Is the dataset complete? (no gaps)
TEST 3: True - Are the resampled and the original dataset of the same size? (so # rows is ok)
TEST 4: True - Is the dataset free from any duplicates?

ONLY CONTINUE IF ALL ARE TRUE!!
--------------------------------
All true. Please, continue. :-)


In [9]:
ohlc_dict = {'open':'first','high':'max','low':'min','close':'last','volume':'sum'}

for resampling_lvl in ['1min','5min','10min','15min','30min']:
    aux = df.resample(resampling_lvl).agg(ohlc_dict)
    aux.reset_index(inplace=True)
#     aux.dropna(inplace=True) gap of ~1hour at the end of July 2018. dropping those for the set to train the mahalanobis matrix in NA
    aux.rename(columns = {'timestamp':'datetime'}, inplace = True)
    print(aux)
    aux.to_csv(RESULT_FILEPATH_PREPROCESSED.replace('.csv','')+'_'+resampling_lvl+'.csv', sep=';', encoding='utf-8', index = False)


                  datetime     open     high      low    close      volume
0      2018-12-31 11:10:00  3790.90  3795.08  3785.81  3792.35  3027362.94
1      2018-12-31 11:11:00  3792.35  3793.46  3788.68  3791.13   796473.41
2      2018-12-31 11:12:00  3791.13  3803.61  3790.45  3803.61   561242.49
3      2018-12-31 11:13:00  3803.61  3807.61  3801.36  3802.51   382831.00
4      2018-12-31 11:14:00  3802.51  3802.51  3798.52  3799.24   202475.13
5      2018-12-31 11:15:00  3799.24  3799.51  3796.49  3798.24   344760.14
6      2018-12-31 11:16:00  3798.24  3810.55  3798.24  3810.55   227546.43
7      2018-12-31 11:17:00  3810.55  3810.55  3803.08  3807.60   287583.92
8      2018-12-31 11:18:00  3807.60  3807.78  3802.58  3805.15   263296.41
9      2018-12-31 11:19:00  3805.15  3806.97  3805.15  3806.31    35409.16
10     2018-12-31 11:20:00  3806.31  3806.89  3805.63  3806.74    89364.82
11     2018-12-31 11:21:00  3806.74  3807.78  3806.53  3806.82    58053.08
12     2018-12-31 11:22:0

                 datetime     open     high      low    close      volume
0     2018-12-31 11:10:00  3790.90  3810.55  3785.81  3806.31  6128981.03
1     2018-12-31 11:20:00  3806.31  3811.85  3804.65  3811.36   943068.85
2     2018-12-31 11:30:00  3811.36  3816.63  3810.79  3811.88   796518.87
3     2018-12-31 11:40:00  3811.88  3812.98  3801.23  3804.37  1584327.22
4     2018-12-31 11:50:00  3804.37  3810.52  3802.05  3804.30  2280229.85
5     2018-12-31 12:00:00  3804.30  3804.30  3796.94  3800.55  2133291.81
6     2018-12-31 12:10:00  3800.55  3808.95  3800.19  3808.95   882448.04
7     2018-12-31 12:20:00  3808.95  3809.26  3804.12  3804.37   764116.73
8     2018-12-31 12:30:00  3804.37  3811.43  3803.59  3807.31   721340.54
9     2018-12-31 12:40:00  3807.31  3807.48  3803.90  3805.75   243489.78
10    2018-12-31 12:50:00  3805.75  3817.67  3803.69  3817.64  1090694.60
11    2018-12-31 13:00:00  3817.64  3823.91  3814.72  3822.30  1175841.47
12    2018-12-31 13:10:00  3822.30  38

                 datetime     open     high      low    close       volume
0     2018-12-31 11:00:00  3790.90  3811.85  3785.81  3811.36   7072049.88
1     2018-12-31 11:30:00  3811.36  3816.63  3801.23  3804.30   4661075.94
2     2018-12-31 12:00:00  3804.30  3809.26  3796.94  3804.37   3779856.58
3     2018-12-31 12:30:00  3804.37  3817.67  3803.59  3817.64   2055524.92
4     2018-12-31 13:00:00  3817.64  3823.91  3810.47  3817.03   2583752.25
5     2018-12-31 13:30:00  3817.03  3818.53  3813.47  3813.73   1991046.79
6     2018-12-31 14:00:00  3813.73  3817.39  3808.66  3812.48   2486255.55
7     2018-12-31 14:30:00  3812.48  3851.91  3812.35  3846.22   6273734.87
8     2018-12-31 15:00:00  3846.22  3858.60  3804.79  3835.77   9003988.60
9     2018-12-31 15:30:00  3835.77  3836.47  3825.82  3827.13   3173413.01
10    2018-12-31 16:00:00  3827.13  3846.61  3814.81  3822.51   5166283.39
11    2018-12-31 16:30:00  3822.51  3823.25  3803.79  3817.41   3097470.90
12    2018-12-31 17:00:00

In [10]:
RESULT_FILEPATH_PREPROCESSED

'C:\\Users\\suare\\data\\analysis\\CRYPTO\\BITCOIN_[2019-01-01_to_2020-07-01].csv'

In [11]:
# all the numbers here and below assume a default time period for ta params of 10 mins averages
# change the other numbers (params of 5min and 20min and theremoval of 20 first mins of the day below) if the mins number is changed.
# in that case change as well the time tag '_10' with the corresponding one.
default_timerange=10
# Set extra timeranges for moving averages
extra_timeranges=[default_timerange/2, default_timerange, default_timerange*2, default_timerange*3, default_timerange*6]

# ###########################################
# Iterate and run list of indicators selected
# All of them produced for 25 prior mins of data 
# ###########################################
for key in list(indicator.keys()):
    # For indicators that only return one column (this will need to be modified depending on the selection of indicators)
    if key in ['ema','sma','trima']:
        for timerange in extra_timeranges:
            df[key+'_'+str(int(timerange))]=indicator[key](df, timeperiod=timerange) 
    elif key not in ['bbands','aroon','stoch','macd','macdfix','stochf']:
        df[key+'_'+str(int(default_timerange))]=indicator[key](df, timeperiod=(default_timerange))
    # Otherwise check the list of columns and return all
    else: 
        key_output=indicator[key](df, timeperiod=(default_timerange)) #, price='close')
        for j in range(0,len(list(key_output.columns))):
            df[key+'_'+key_output.columns[int(j)]]=key_output[key_output.columns[j]]
            
df.head()

,close,high,low,open,time,volumefrom,volume,timestamp,sma_5,sma_10,...,mom_10,stoch_slowk,stoch_slowd,macd_macd,macd_macdsignal,macd_macdhist,rsi_10,willr_10,adosc_10,cci_10
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-12-31 11:10:00,3792.35,3795.08,3785.81,3790.90,1546254600,800.98,3027362.94,2018-12-31 11:10:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 11:11:00,3791.13,3793.46,3788.68,3792.35,1546254660,210.10,796473.41,2018-12-31 11:11:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 11:12:00,3803.61,3803.61,3790.45,3791.13,1546254720,148.37,561242.49,2018-12-31 11:12:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 11:13:00,3802.51,3807.61,3801.36,3803.61,1546254780,101.42,382831.00,2018-12-31 11:13:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-12-31 11:14:00,3799.24,3802.51,3798.52,3802.51,1546254840,53.66,202475.13,2018-12-31 11:14:00,3797.768,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def set_label(row):
    if row['close_t+1'] > row['close']: return 1
    else: return 0

# One minute ahead closing price
df['close_t+1']=df['close'].shift(-1)

# Creating label/y to be predicted / independent (predicted) feature 'y' 
df['label'] = df.apply(set_label, axis=1)
df.head()

,close,high,low,open,time,volumefrom,volume,timestamp,sma_5,sma_10,...,stoch_slowd,macd_macd,macd_macdsignal,macd_macdhist,rsi_10,willr_10,adosc_10,cci_10,close_t+1,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2018-12-31 11:10:00,3792.35,3795.08,3785.81,3790.90,1546254600,800.98,3027362.94,2018-12-31 11:10:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3791.13,0
2018-12-31 11:11:00,3791.13,3793.46,3788.68,3792.35,1546254660,210.10,796473.41,2018-12-31 11:11:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3803.61,1
2018-12-31 11:12:00,3803.61,3803.61,3790.45,3791.13,1546254720,148.37,561242.49,2018-12-31 11:12:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3802.51,0
2018-12-31 11:13:00,3802.51,3807.61,3801.36,3803.61,1546254780,101.42,382831.00,2018-12-31 11:13:00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3799.24,0
2018-12-31 11:14:00,3799.24,3802.51,3798.52,3802.51,1546254840,53.66,202475.13,2018-12-31 11:14:00,3797.768,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3798.24,0


In [13]:
# Filter out irrelevant dates once the dataset is complete.
df=df[(df['timestamp'] > start_date) & (df['timestamp'] <= end_date+" 23:59:59")]

In [14]:
# Select columns for output
columns_selected=['timestamp',
                 'rsi_10','willr_10','adosc_10','macd_macd' ,'cci_10','mom_10',
                 'stoch_slowk','stoch_slowd',
                 'ema_5','ema_10','ema_20','ema_30','ema_60',
                 'sma_5','sma_10','sma_20','sma_30','sma_60',
                 'label']

# Export processed data
output = pd.DataFrame(df, columns=columns_selected)
output.to_csv(RESULT_FILEPATH_PROCESSED, sep=';', encoding='utf-8', index = False)

# Printing classes distributions
print("In crypto: "+CURRENCY)
label_zero=len(df[df['label'] == 0])
label_one=len(df[df['label'] == 1])
#print(label_zero)
#print(label_one)
print("0 in "+str(float(label_zero)/(label_one+label_zero))+"%")
print("1 in "+str(float(label_one)/(label_one+label_zero))+"%")

In crypto: BITCOIN
0 in 0.5045786418071321%
1 in 0.4954213581928679%


In [107]:
# Class distribution 2 months before Q3 2018

#In crypto: LITECOIN
#0 in 0.5770947176684882%
#1 in 0.42290528233151187%
##
#In crypto: BITCOIN
#0 in 0.5248178506375227%
#1 in 0.4751821493624772%

#In crypto: DASH
#0 in 0.5934767759562841%
#1 in 0.4065232240437158%

# 
#In crypto: ETHEREUM
#0 in 0.511976320582878%
#1 in 0.48802367941712205%
# 
#In crypto: RIPPLE
#0 in 0.5984631147540984%
#1 in 0.40153688524590164%
# 
#In crypto: MONERO
#0 in 0.5995901639344262%
#1 in 0.4004098360655738%

# ########################################

# Class distribution in Q3 2018 to Q2 2019 (included)
#In crypto: MONERO
#0 in 0.7447983257229832%
#1 in 0.25520167427701673%
# 
#In crypto: RIPPLE
#0 in 0.6268797564687976%
#1 in 0.37312024353120243%
# 
#In crypto: ETHEREUM
#0 in 0.5466057838660578%
#1 in 0.45339421613394215%
# 
#In crypto: DASH
#0 in 0.7087347792998477%
#1 in 0.2912652207001522%
# 
#In crypto: BITCOIN
#0 in 0.4984931506849315%
#1 in 0.5015068493150685%
# 
#In crypto: LITECOIN
#0 in 0.6055745814307458%
#1 in 0.39442541856925417%


# Class distribution in Q3 2018
# In crypto: MONERO
# 0 in 0.689779589372%
# 1 in 0.310220410628%
# 
# In crypto: RIPPLE
# 0 in 0.641070350242%
# 1 in 0.358929649758%
# 
# In crypto: ETHEREUM
# 0 in 0.535235507246%
# 1 in 0.464764492754%
# 
# In crypto: DASH
# 0 in 0.639447463768%
# 1 in 0.360552536232%
# 
# In crypto: BITCOIN
# 0 in 0.503985507246%
# 1 in 0.496014492754%
# 
# In crypto: LITECOIN
# 0 in 0.62444897343%
# 1 in 0.37555102657%

# class distribution in July 2019

In crypto: LITECOIN
0 in 0.5386200716845878%
1 in 0.4613799283154122%

In crypto: RIPPLE
0 in 0.613978494623656%
1 in 0.3860215053763441%

In crypto: MONERO
0 in 0.6600806451612903%
1 in 0.33991935483870966%

In crypto: ETHEREUM
0 in 0.5233870967741936%
1 in 0.47661290322580646%

In crypto: DASH
0 in 0.6907482078853047%
1 in 0.30925179211469533%

In crypto: BITCOIN
0 in 0.5022849462365592%
1 in 0.4977150537634409%

SyntaxError: invalid syntax (<ipython-input-107-c738fb085f89>, line 83)